In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
import random as rd
from textblob import TextBlob
import requests
import time
from random import randint

data=pd.read_csv("Questions.csv",encoding= 'iso-8859-1')


In [2]:
START_SPELL_CHECK="<span class=\"spell\">Showing results for</span>"
END_SPELL_CHECK="<br><span class=\"spell_orig\">Search instead for"

HTML_Codes = (("'", '&#39;'),('"', '&quot;'),('>', '&gt;'),('<', '&lt;'),('&', '&amp;'),)

def spell_check(s):
    q = '+'.join(s.split())
    time.sleep(  randint(0,2) ) #relax and don't let google be angry
    r = requests.get("https://www.google.co.uk/search?q="+q)
    content = r.text
    start=content.find(START_SPELL_CHECK) 
    if ( start > -1 ):
        start = start + len(START_SPELL_CHECK)
        end=content.find(END_SPELL_CHECK)
        search= content[start:end]
        search = re.sub(r'<[^>]+>', '', search)
        for code in HTML_Codes:
            search = search.replace(code[1], code[0])
        search = search[1:]
    else:
        search = s
    return search ;

searches = ["i havv a good boyfriend."]
for search in searches:
    speel_check_search= spell_check(search)
    print (search+"->" + speel_check_search)

i havv a good boyfriend.->i have a good boyfriend.


In [3]:
data.tail(3)


,Id,OwnerUserId,CreationDate,Score,Title,Body
607279,40143267,4064680,2016-10-19T23:44:07Z,0,Updating an ManyToMany field with Django rest,<p>I'm trying to set up this API so I can use ...
607280,40143338,7044980,2016-10-19T23:52:27Z,2,Most possible pairs,"<p>Given a list of values, and information on ..."
607281,40143365,4133131,2016-10-19T23:55:49Z,0,Theano operations returning odd results,<p>So I'm trying to learn how to use Theano an...


In [4]:
data.shape

(607282, 6)

In [5]:
data.describe()

,Id,OwnerUserId,Score
count,607282.000000,601070.000000,607282.000000
mean,23719604.527590,2519594.996418,2.283137
std,11247153.677061,1910375.393429,19.285578
min,469.000000,25.000000,-44.000000
25%,14855187.250000,853934.000000,0.000000
50%,25318970.000000,2107677.000000,1.000000
75%,33588233.500000,3991164.000000,2.000000
max,40143365.000000,7044992.000000,5524.000000


In [18]:
from html.parser import HTMLParser
import validators
    
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)
    
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

stop_w = set(stopwords.words("english")) #stop words in English
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}
def clean(s):
    if isinstance(s, unicode):
        code_blocks = re.findall(r"<code>([^<]*)</code>",s)
        comments = re.findall(r"#([^<]*)\n",s)
        codes = " ".join(code_blocks)
        real_comments = ' '.join([comments[i] for i in range(len(comments)) if comments[i] in  " ".join(code_blocks)])# extract comments in codes
        s = re.sub(r"(<code>)[^<]+(</code>)",r"\1 \2",s)# remove code blocks.
        url = re.findall(r'<a [\w\W]+>(.*)</a>',s)
        for i in range(len(url)):
            if validators.url(url[i]):
                   s = s.replace(url[i]," ")# remove link labels
        s = s + real_comments
        s = strip_tags(s) #remove html tags
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(","," ") 
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("..",".")
        s = s.replace("\\"," ")
        s = s.replace("."," . ")
        s = s.replace(":"," ")
        s = s.replace("#"," ")
        s = s.replace("("," ")
        s = s.replace(")"," ")
        s = s.replace('"'," ")
        s = s.replace('='," ")
        s = s.replace('!'," ")
        s = s.replace("\n"," ")
        s = s.replace("\r"," ")
        s = s.replace("\\n"," ")
        s = s.replace("\\r"," ")
        s = s.replace("\t"," ")
        s = s.replace("\\t"," ")
        s = s.replace("\f"," ")
        s = s.replace("\\f"," ")
        s = s.replace(" os x "," osx ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = s.replace(" os x "," osx ") # mac os x -> mac osx
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = re.sub("\'"," ",s) # 's->" " 
        s = re.sub(r" +"," ",s) # "  "-> " "
        s = s.replace(" . "," ")
        s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        
        #s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()

        return s
    
    elif isinstance(s, str):
        code_blocks = re.findall(r"<code>([^<]*)</code>",s)
        comments = re.findall(r"#([^<]*)\n",s)
        codes = " ".join(code_blocks)
        real_comments = ' '.join([comments[i] for i in range(len(comments)) if comments[i] in  " ".join(code_blocks)])# extract comments in codes
        s = re.sub(r"(<code>)[^<]+(</code>)",r"\1 \2",s)# remove code blocks.
        url = re.findall(r'<a [\w\W]+>(.*)</a>',s)
        for i in range(len(url)):
            if validators.url(url[i]):
                   s = s.replace(url[i]," ")# remove link labels
        s = s + real_comments
        s = strip_tags(s)
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(","," ")
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("..",".")
        s = s.replace("\\"," ")
        s = s.replace("."," . ")
        s = s.replace(":"," ")
        s = s.replace("#"," ")
        s = s.replace("("," ")
        s = s.replace(")"," ")
        s = s.replace('"'," ")
        s = s.replace('='," ")
        s = s.replace('!'," ")
        s = s.replace("\n"," ")
        s = s.replace("\r"," ")
        s = s.replace("\\n"," ")
        s = s.replace("\\r"," ")
        s = s.replace("\t"," ")
        s = s.replace("\\t"," ")
        s = s.replace("\f"," ")
        s = s.replace("\\f"," ")
        s = s.replace(" os x "," osx ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = re.sub("\'"," ",s) 
        s = re.sub(r" +"," ",s) # "  "-> " "
        s = s.replace(" . "," ")
        s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = re.sub(r'\'',' ',s)
        #s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        return s
    
    else:
        return "null"

In [19]:
i=randint(0,607282)
i

354185

In [20]:
data['Body'][i]

u"<p>I have a case that I need to extract cookies from a redirect response(login page, for example).</p>\n\n<p>I have read the doc, even the source code, but still can't find a way to do this.</p>\n\n<p>By default, <code>AsyncHTTPClient</code> will follow the redirect and return the redirected response, which didn't contain <code>Set-Cookie</code> thus useless. If given option <code>follow_redirects=False</code>, it will throw an <code>HTTPError</code> just because the response code isn't 200. I also tried the blocking <code>HTTPClient</code> class, it's same.</p>\n\n<p>So, what's the right way to do this? I think it's common way that server will send <code>Set-Cookie</code> header along side with a <code>Location</code> header when a user login successfully.</p>\n\n<p>Could it be considered as a bug? Or, design deficiency?</p>\n"

In [21]:
clean(data['Body'][i])

u'case need extract cookies redirect response login page example read doc even source code still find way default follow redirect return redirected response contain thus useless given option throw response code 200 also tried blocking class right way think common way server send header along side header user login successfully could considered bug design deficiency '

In [ ]:
all_cleaned_data=data
all_cleaned_data['Title']=all_cleaned_data['Title'].map(lambda x:clean(x)) #how to use clean function
all_cleaned_data['Body']=all_cleaned_data['Body'].map(lambda x:clean(x))